# INTRODUCCIÓN

Se ha decidido abrir un café regentado por robots en Los Ángeles y se debe atraer a inversionistas. Con datos procedentes de fuentes abiertas sobre restaurantes en Los Ángeles hay que decidir si el café seguirá teniendo éxito cuando la novedad de los camareros robot desaparezca.

Para esto se van a preparar los datos para luego realizar un análisis. Luego se preparara una presentación con el debido formato y las conclusiones.

# TABLA DE CONTENIDO

# 1. Preparación de datos
# 2. Análisis de datos
# 3. Presentación
# 4. Conclusión y recomendaciones

In [33]:
#importando los paquetes
import pandas as pd
from plotly import graph_objects as go
import plotly.express as px
import seaborn as sns


#import datetime as dt
#import numpy as np
import matplotlib.pyplot as plt
#import scipy.stats as stats


## PREPARACIÓN DE DATOS

In [34]:
#guardando y mostrando el dataset
try:
    rest_data = pd.read_csv("/datasets/rest_data_us.csv")
except:
    rest_data = pd.read_csv("rest_data_us.csv")
    
rest_data.head()

,id,object_name,address,chain,object_type,number
0,11786,HABITAT COFFEE SHOP,3708 N EAGLE ROCK BLVD,False,Cafe,26
1,11787,REILLY'S,100 WORLD WAY # 120,False,Restaurant,9
2,11788,STREET CHURROS,6801 HOLLYWOOD BLVD # 253,False,Fast Food,20
3,11789,TRINITI ECHO PARK,1814 W SUNSET BLVD,False,Restaurant,22
4,11790,POLLEN,2100 ECHO PARK AVE,False,Restaurant,20


In [35]:
#observando el tipo de datos
rest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9651 entries, 0 to 9650
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           9651 non-null   int64 
 1   object_name  9651 non-null   object
 2   address      9651 non-null   object
 3   chain        9648 non-null   object
 4   object_type  9651 non-null   object
 5   number       9651 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 452.5+ KB


Valores ausentes: Existen 3 valores ausentes en la columna "chain" que se deben trabajar.

Tipo de dato: La columna "chain" debe ser tipo bool.

### Verificando valores duplicados

In [36]:
#verificando si hay valores duplicados
rest_data[["object_name","address"]].duplicated().sum()

0

### Verificando y rellenando valores ausentes

In [37]:
#contando los valores en "chain"
rest_data["chain"].value_counts()

False    5972
True     3676
Name: chain, dtype: int64

In [38]:
#verificando los valores ausentes en "chain"
rest_data[rest_data["chain"].isna()]

,id,object_name,address,chain,object_type,number
7408,19194,TAQUERIA LOS 3 CARNALES,5000 E WHITTIER BLVD,NaN,Restaurant,14
7523,19309,JAMMIN JIMMY'S PIZZA,1641 FIRESTONE BLVD,NaN,Pizza,1
8648,20434,THE LEXINGTON THEATER,129 E 3RD ST,NaN,Restaurant,35


In [39]:
#creando un dataset con la cantidad de locales de cada restaurante
rest_quantity = rest_data.groupby("object_name").agg({"id":"count"}).sort_values(by="id", ascending=False).reset_index()
rest_quantity.columns = ["name", "count"]
rest_quantity.head()

,name,count
0,THE COFFEE BEAN & TEA LEAF,47
1,SUBWAY,31
2,DOMINO'S PIZZA,15
3,KENTUCKY FRIED CHICKEN,14
4,WABA GRILL,14


In [40]:
#calculando la cantidad de restaurantes que tienen más de un local en el dataset
chain_list = rest_quantity[rest_quantity["count"] > 1]["name"]
chain_list.count()

524

In [41]:
#calculando la cantidad de restaurantes que solo tienen un local en el dataset
no_chain_list = rest_quantity[rest_quantity["count"] == 1]["name"]
no_chain_list = list(no_chain_list)
len(no_chain_list)

8148

No hay forma de determinar a través de los datos si los datos faltantes en "chain" son False o no. Hay algunos restaurantes que si pertenecen a una cadena pero que solo hay un restaurant en la base de datos, por lo que no se puede decir que si solo hay uno en el dataset, significa que no pertenece a una cadena. Esto significa que es mejor eliminar los datos faltantes para que no interfieran en las conclusiones. La eliminación de estos datos no afectará tanto ya que solo son 3 datos, es decir el 0.03% de los datos. 

In [42]:
#obteniendo la información del dataset
rest_data = rest_data.dropna()
rest_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9648 entries, 0 to 9650
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           9648 non-null   int64 
 1   object_name  9648 non-null   object
 2   address      9648 non-null   object
 3   chain        9648 non-null   object
 4   object_type  9648 non-null   object
 5   number       9648 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 527.6+ KB


In [43]:
#verificando los valores ausentes en "chain"
rest_data[rest_data["chain"].isna()]

,id,object_name,address,chain,object_type,number


### Cambiando los tipos de datos

In [44]:
rest_data["chain"] = rest_data["chain"].astype("bool")
rest_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9648 entries, 0 to 9650
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           9648 non-null   int64 
 1   object_name  9648 non-null   object
 2   address      9648 non-null   object
 3   chain        9648 non-null   bool  
 4   object_type  9648 non-null   object
 5   number       9648 non-null   int64 
dtypes: bool(1), int64(2), object(3)
memory usage: 461.7+ KB


## ANÁLISIS DE DATOS

### Proporciones de tipos de establecimientos

In [45]:
#agrupando los datos para graficar
type_list = rest_data.groupby("object_type").agg({"id":"count"}).sort_values(by="id", ascending=False).reset_index()
type_list.columns = ["type", "count"]
type_list

,type,count
0,Restaurant,7253
1,Fast Food,1066
2,Cafe,435
3,Pizza,319
4,Bar,292
5,Bakery,283


In [46]:
#graficando la proporción de los tipos de establecimientos
type_rest = list(type_list["type"])
count_rest = list(type_list["count"])
fig = go.Figure(data = [go.Pie(labels = type_rest, values = count_rest, title="Tipos de establecimientos")])
fig.show()

### Proporciones de tipos de establecimientos que pertenecen o no a una cadena

In [47]:
#contando los valores en "chain"
chain_ratio = rest_data.groupby("chain").agg({"id":"count"}).reset_index()
chain_ratio.columns = ["is_chain", "quantity"]
chain_ratio

,is_chain,quantity
0,False,5972
1,True,3676


In [48]:
#graficando la proporción de establecimientos en cadena e independientes
is_chain = list(chain_ratio["is_chain"])
quantity = list(chain_ratio["quantity"])
fig = go.Figure(data = [go.Pie(labels = is_chain, values = quantity, title="Proporción de establecimientos en cadena")])
fig.show()

In [49]:
#agrupando para graficar
type_list = rest_data.groupby(["chain", "object_type"]).agg({"id":"count"}).sort_values(by="id", ascending=False).reset_index()
type_list.columns = ["chain", "type", "count"]
type_list

,chain,type,count
0,False,Restaurant,4961
1,True,Restaurant,2292
2,True,Fast Food,605
3,False,Fast Food,461
4,True,Bakery,283
5,True,Cafe,266
6,False,Bar,215
7,False,Cafe,169
8,False,Pizza,166
9,True,Pizza,153


In [50]:
#graficando la proporción de los tipos de establecimientos por grupo (si son cadena o no)
fig = px.bar(type_list, x="type", y="count", color="chain", title="Cantidad de establecimientos diferenciado por grupo (cadena o no cadena)")
fig.show()

### ¿Qué tipo de establecimiento es habitualmente una cadena?

Como se ve en el gráfico anterior, los establecimientos que son bakery habitualmente son cadena, de hecho no hay información sobre bakerys que no pertenezcan a una cadena.

Además, los establecimientos de fast food y de café tambien en su mayoría son parte de una cadena, aunque en menor proporción que bakery. 

### ¿Qué caracteriza a las cadenas?

In [51]:
#creando un dataset solo con los datos pertenecientes a las cadenas
chain_list = rest_data.query('chain == True')
chain_list.columns = ["id", "object_name", "address", "chain", "object_type", "seats"]
chain_list.head()

,id,object_name,address,chain,object_type,seats
8,11794,ABC DONUTS,3027 N SAN FERNANDO RD UNIT 103,True,Fast Food,1
10,11796,EL POLLO LOCO,5319 W SUNSET BLVD,True,Restaurant,38
11,11797,POONG NYUN BAKERY,928 S WESTERN AVE STE 109,True,Bakery,17
12,11798,EMC RESTAURANT GROUP LLC,3500 W 6TH ST STE 101,True,Restaurant,133
14,11800,CUSCATLECA BAKERY,2501 W SUNSET BLVD #A,True,Bakery,26


In [52]:
#graficando un histograma
fig = px.histogram(chain_list, x="seats", nbins = 20, title="Distribución del número de asientos en establecimientos en cadena")
fig.show()

Se observa en el histograma que en general los establecimientos de cadena tienen pocos asientos, el mayor tramo está entre 0 y 19 asientos, esto se puede deber a que las cadenas priorizan la rotación rápida de los clientes.

In [53]:
#obteniendo información sobre la cantidad de establecimientos
chain_list["seats"].describe()

count    3676.000000
mean       39.694233
std        43.437212
min         1.000000
25%        13.000000
50%        25.000000
75%        44.000000
max       229.000000
Name: seats, dtype: float64

Solo el 25% de los establecimientos tienen más de 44 asientos.

In [54]:
#graficando un histograma
fig = px.histogram(chain_list, x="object_name", nbins = 50, title="Distribución del número de locales por cadena")
fig.show()

Del histograma anterior se puede observar que en general las cadenas se caracterizan por tener pocos establecimientos (menos de 10).

Como conclusión a esta sección, se desprende que las cadenas se caracterizan por tener pocos establecimientos y menos de 50 asientos.

### Número de asientos promedio para cada tipo de restaurante

In [55]:
#cambiando el nombre de las columnas a graficar
type_list = rest_data
type_list.columns = ["id", "object_name", "address", "chain", "establishments", "seats"]

In [56]:
#graficando el numero de asientos para cada tipo de establecimiento
px.box(rest_data, x="establishments", y="seats", title="Asientos promedio para cada tipo de establecimiento")

Los restaurantes y los bares son los establecimientos con mayor cantidad de asientos en promedio.

### Mejores calles para restaurantes

In [57]:
#extrayendo los nombres de las calles en otra columna
rest_data['address_2'] = rest_data['address'].str.extract(pat='([a-zA-Z][\w ]+(?!STE|UNIT|MEZZ|[\w ]+))')
rest_data['address_2'] = rest_data['address_2'].str.strip()
rest_data.head()

,id,object_name,address,chain,establishments,seats,address_2
0,11786,HABITAT COFFEE SHOP,3708 N EAGLE ROCK BLVD,False,Cafe,26,N EAGLE ROCK BLVD
1,11787,REILLY'S,100 WORLD WAY # 120,False,Restaurant,9,WORLD WAY
2,11788,STREET CHURROS,6801 HOLLYWOOD BLVD # 253,False,Fast Food,20,HOLLYWOOD BLVD
3,11789,TRINITI ECHO PARK,1814 W SUNSET BLVD,False,Restaurant,22,W SUNSET BLVD
4,11790,POLLEN,2100 ECHO PARK AVE,False,Restaurant,20,ECHO PARK AVE


In [58]:
#arreglando algunos datos
rest_data["address_2"] = rest_data["address_2"].replace("ABBOT KINNEY", "ABBOT KINNEY BLVD")
rest_data["address_2"] = rest_data["address_2"].replace("hollywood BLVD", "HOLLYWOOD BLVD")
rest_data["address_2"] = rest_data["address_2"].replace("Z00 DR", "ZOO DR")

In [59]:
#agrupando para obtener cantidad de restaurantes por calle
count_streets = rest_data.groupby("address_2").agg({"id":"count"}).sort_values(by="id", ascending=False).reset_index()
count_streets.columns = ["street", "restaurants"]
best_streets = count_streets.head(10)
best_streets

,street,restaurants
0,W SUNSET BLVD,327
1,W PICO BLVD,313
2,WILSHIRE BLVD,245
3,SANTA MONICA BLVD,211
4,S WESTERN AVE,208
5,HOLLYWOOD BLVD,204
6,W 3RD ST,183
7,S FIGUEROA ST,178
8,S VERMONT AVE,177
9,W OLYMPIC BLVD,163


In [60]:
#graficando las mejores calles
fig = px.bar(best_streets, x="street", y="restaurants", title="10 calles con más restaurantes")
fig.update_xaxes(tickangle=45)
fig.show()

### Peores calles para restaurantes

In [61]:
#conteo de las calles con solo 1 restaurante
count_streets[count_streets["restaurants"]==1]["street"].count()

1211

### Distribución del número de asientos en las mejores calles

In [62]:
#creando una lista de las mejores calles
best_streets_list = list(best_streets["street"])
best_streets_list

['W SUNSET BLVD',
 'W PICO BLVD',
 'WILSHIRE BLVD',
 'SANTA MONICA BLVD',
 'S WESTERN AVE',
 'HOLLYWOOD BLVD',
 'W 3RD ST',
 'S FIGUEROA ST',
 'S VERMONT AVE',
 'W OLYMPIC BLVD']

In [63]:
#obteniendo el dataset solo de las mejores calles
best_data = rest_data.query('address_2 in @best_streets_list')
best_data

,id,object_name,address,chain,establishments,seats,address_2
2,11788,STREET CHURROS,6801 HOLLYWOOD BLVD # 253,False,Fast Food,20,HOLLYWOOD BLVD
3,11789,TRINITI ECHO PARK,1814 W SUNSET BLVD,False,Restaurant,22,W SUNSET BLVD
10,11796,EL POLLO LOCO,5319 W SUNSET BLVD,True,Restaurant,38,W SUNSET BLVD
14,11800,CUSCATLECA BAKERY,2501 W SUNSET BLVD #A,True,Bakery,26,W SUNSET BLVD
16,11802,HMS BOUNTY,3357 WILSHIRE BLVD,False,Restaurant,147,WILSHIRE BLVD
...,...,...,...,...,...,...,...
9628,21414,KIFF KAFE,12217 W PICO BLVD,False,Restaurant,29,W PICO BLVD
9635,21421,THE TEA & COFFEE EXCHANGE,6801 HOLLYWOOD BLVD # 120,False,Cafe,4,HOLLYWOOD BLVD
9639,21425,MCDONALD'S,1800 S WESTERN AVE,True,Fast Food,135,S WESTERN AVE
9645,21431,SAINT MARC,10250 SANTA MONICA BLVD # 1025,False,Restaurant,225,SANTA MONICA BLVD


In [64]:
#graficando el histograma
fig = px.histogram(best_data, x="seats", nbins = 20, title="Distribución del número de asientos en las mejores calles")
fig.show()

La tendencia de la distribución del número de asiento en las mejores calles no varía mucho a las demás. Existen muchos más establecimientos con pocos asientos.

## PRESENTACIÓN

Presentation: <https://drive.google.com/file/d/13RcWkGpM8sMdG-xJN1NuTQymm7Opv9Pz/view?usp=sharing>

## CONCLUSIÓN Y RECOMENDACIONES

El estudio de la base de datos presentada determinó que la competencia de cafeterías es bastante pequeña en comparación con otros tipos de establecimientos como los restaurantes, por lo que llamará bastante la atención la entrada al mercado de una cafetería regentada por robots.

Como recomendación, la cafetería debe formar parte de una cadena, y los locales deben estar ubicados en las calles más populares, ya que es ahí donde concurrirá mucha gente. También sería importante mantener una baja cantidad de asientos, ya que así se potencia la rápida rotación de los clientes. 